<a href="https://colab.research.google.com/github/junghyun9108/2025_DL/blob/main/12%EA%B0%95_%EC%9E%90%EA%B8%B0%EC%A1%B0%EC%A7%81%ED%99%94%EC%A7%80%EB%8F%84_SOM_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
 import numpy as np
 import matplotlib.pyplot as plt
 import matplotlib.animation as animation
 %matplotlib notebook

In [31]:
# SOM 클래스 정의
class SOM:
    def __init__(self, width, height, input_dim):
        self.width = width
        self.height = height
        self.input_dim = input_dim
        self.weights = np.random.rand(height, width, input_dim)
        self.learning_rate = 0.02
        self.radius = max(width, height)/2   # 이웃의 범위를 지정

    def _find_bmu(self, input_vec):
        distance = np.sum((self.weights - input_vec) ** 2, axis=2)
        return np.unravel_index(distance.argmin(), distance.shape) #거리가 가장 작은 유닛의 인덱스를 리턴

    def _update_weights(self, input_vec, bmu, iteration, total_iterations):  #가중치 업데이트 함수
        learning_rate = self.learning_rate * np.exp(-iteration / total_iterations)
        radius_decay = self.radius * np.exp(-iteration / total_iterations)

        for x in range(self.width):     # 각 뉴런들과 BMU 간의 위치상의 거리를 하나씩 계산
            for y in range(self.height):
                dist_to_bmu = np.sqrt((x-bmu[1]) ** 2 + (y-bmu[0]) ** 2 )
                if dist_to_bmu <= radius_decay: # 그 격자상의 거리상 그 뉴런이 이웃 반경안에 있는 뉴런이라면
                    influence = np.exp(-dist_to_bmu **2 / (2 * (radius_decay ** 2)))
                    self.weights[y, x, :] += learning_rate * influence * (input_vec - self.weights[y, x, :])

    def train(self, data, num_iterations):
        weight_maps = []
        for i in range(num_iterations):
            for input_vec in data:
                bmu = self._find_bmu(input_vec)
                self._update_weights(input_vec, bmu, i , num_iterations)
            weight_maps.append(np.copy(self.weights))
        return weight_maps

In [32]:
# SOM 객체 생성 및 학습 데이터 준비
som = SOM(20, 20, 3)
data = np.random.rand(100, 3)
num_iterations = 20

In [33]:
#학습 실행 및 가중치 기록
weight_maps = som.train(data, num_iterations)

In [42]:
# 가중치 변화 시각화를 위한 애니메이션 함수 정의
def animate(i):
    plt.imshow(weight_maps[i])
    plt.title(f'Iterations: {i+1}')
    plt.axis('off')

In [43]:
# 애니메이션 생성 및 표시
fig = plt.figure(figsize = (10,10))
ani = animation.FuncAnimation(fig, animate, frames=num_iterations, interval = 100, repeat= False)
plt.show()

<IPython.core.display.Javascript object>

In [44]:
from IPython.display import HTML
ani = animation.FuncAnimation(fig, animate, frames=num_iterations, interval=100, repeat=False)
HTML(ani.to_jshtml())  # 애니메이션을 HTML로 변환하여 표시


/usr/local/lib/python3.11/dist-packages/matplotlib/animation.py:908: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(
